# CARGA DE DADOS DO BIRH
---
O BIRH é a base de dados integrada de RH da Petrobras.

Esta base de dados será consultada para extração de dados corporativos não sensíveis dos usuários


In [1]:
import os
import os.path
import cx_Oracle
import sqlalchemy
import pandas as pd
import duckdb

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
oracle_host       = os.environ['ORACLE_HOST']
oracle_port       = os.environ['ORACLE_PORT']
oracle_service    = os.environ['ORACLE_SERVICE']
oracle_user       = os.environ['ORACLE_USER']
oracle_password   = os.environ['ORACLE_PASSWORD']
oracle_home       = os.environ['ORACLE_HOME']

A biblioteca cx_Oracle requer a instação de um Oracle Client contendo bibliotecas C necessária para comunicação com o servidor.

A maneira mais fácil de atender esse requisito é por meio da instalação do Oracle Instant Client, que pode ser baixada e descompactada na pasta do próprio projeto.

Segue link abaixo:

https://www.oracle.com/br/database/technologies/instant-client/winx64-64-downloads.html

Após instalação, é necessário indicar na instalação da biblioteca onde a mesma encontra-se

In [3]:
cx_Oracle.init_oracle_client(lib_dir=oracle_home)

A consulta abaixo retorna a lista de usuários com chave ativa, trazendo tanto funcionários próprios como concusrsados.

Nenhuma informação sensível sobre os mesmos está sendo retornada, nem mesmo o nome nome das pessoas

In [4]:
SQL_DADOS_BIRH = """
    SELECT  a.fotr_cd_chave                                     AS chave_usuario
    ,       UPPER(a.tift_nm_tipo_forca_trabalho)                AS tipo_usuario
    ,       a.lota_cd_centro_custo                              AS centro_custo
    ,       NVL(REGEXP_SUBSTR(
                unor_sg_unidade_organizacional
            ,   '(^[^/]+)'
            ,   1     -- Start at the first character
            ,   1     -- Get the first match
            ,   NULL  -- Case sensitive
            ,   1     -- Return the first capture group
            ), 'n/a')                                           AS lotacao_topo
    ,       NVL(UPPER(a.unor_sg_unidade_organizacional), 'n/a') AS sigla_lotacao
    ,       NVL(UPPER(a.unor_nm_unidade_organizacional), 'n/a') AS nome_lotacao
    ,       NVL(UPPER(a.carg_nm_cargo), 'n/a')                  AS cargo
    ,       NVL(UPPER(a.enfa_nm_enfase), 'n/a')                 AS enfase
    ,       NVL(UPPER(a.func_nm_funcao), 'n/a')                 AS funcao
    ,       UPPER(a.sind_nm_sindicato)                          AS sindicato
    ,       UPPER(a.arrh_nm_area_rh)                            AS area_rh
    ,       UPPER(a.ftre_nm_imovel_subarea_rh)                  AS imovel
    ,       UPPER(a.lone_nm_local_negocio)                      AS local_negocio
    ,       UPPER(a.grap_nm_grupo_ativ_prest_serv)              AS grupo_prestacao_servico
    ,       UPPER(a.retr_ds_regime_trab_permanente)             AS regime_trabalho
    ,       UPPER(a.emgp_nm_empresa_contrato)                   AS empresa_contrato
    ,       UPPER(a.prtl_in_tipo_localizacao)                   AS tipo_localizacao
    ,       a.pcse_in_status_cracha                             AS status_cracha
    ,       a.pcse_in_situacao_cracha                           AS situacao_cracha
    FROM    tcrp.MV_BIRH_FOTR_PRSE_VIGENTE a -- MVIEW do BIRH Força de Trabalho & Prestadores de Serviços Vigentes
    WHERE   a.emgp_cd_sap_empresa                   = '1000'
    AND     a.fotr_in_estado_chave                  = 'A'
    AND     a.siat_cd_situacao                      = 'A'
"""

SQL_OBJETOS_CONTRATOS = """
    SELECT  a.unor_sg_unid_org_ger              AS sigla_lotacao
    ,       a.cont_tx_objeto                    AS objeto_contrato
    FROM    tcrp.mv_bictr_contrato_petr_k a
    WHERE   a.cont_in_ativo                     = 'S'
    AND     a.cont_tx_objeto                    IS NOT NULL
    AND     a.unor_sg_unid_org_ger              IS NOT NULL
    --
    UNION
    --
    SELECT  b.unor_sg_unid_org_fiscal           AS sigla_lotacao            
    ,       a.cont_tx_objeto                    AS objeto_contrato
    FROM    tcrp.mv_bictr_contrato_petr_k a
            --
            INNER JOIN tcrp.mv_bictr_fiscal_petr_k b
            ON	a.emgp_cd_sap_empresa           = b.emgp_cd_sap_empresa
            AND	a.cont_cd_contrato_sap          = b.cont_cd_contrato_sap
            AND	b.fico_in_manutencao            = 'I'
            --
    WHERE   a.cont_in_ativo                     = 'S'
    AND     a.cont_tx_objeto                    IS NOT NULL
    AND     b.unor_sg_unid_org_fiscal           IS NOT NULL
    --
    ORDER   BY sigla_lotacao
    ,       objeto_contrato
"""

SQL_CURSOS = """
    SELECT  /*+ ALL_ROWS */
            ustr_cd_chave_usuario                       AS chave_usuario
    ,       UPPER(itct_nm_titulo_item_curso)            AS curso
    FROM    tcrp.mv_birh_item_usuario a
"""

Estabelecimento da conexão usando sqlalchemy. A função **pandas.read_sql** utiliza sqlalchemy para conversar com qualquer banco de dados (além do sqlite)

In [5]:
oracle_dsn        = cx_Oracle.makedsn(oracle_host, oracle_port , oracle_service).replace('SID', 'SERVICE_NAME')
oracle_connstring = f'oracle://{oracle_user}:{oracle_password}@{oracle_dsn}'
engine            = sqlalchemy.create_engine(oracle_connstring)
conn              = engine.connect()

Extração do conjunto de dados

In [6]:
_birh_df = pd.read_sql(SQL_DADOS_BIRH, conn)
_birh_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91830 entries, 0 to 91829
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            91798 non-null  object
 1   tipo_usuario             91830 non-null  object
 2   centro_custo             39913 non-null  object
 3   lotacao_topo             91830 non-null  object
 4   sigla_lotacao            91830 non-null  object
 5   nome_lotacao             91830 non-null  object
 6   cargo                    91830 non-null  object
 7   enfase                   91830 non-null  object
 8   funcao                   91830 non-null  object
 9   sindicato                39415 non-null  object
 10  area_rh                  91296 non-null  object
 11  imovel                   91296 non-null  object
 12  local_negocio            91293 non-null  object
 13  grupo_prestacao_servico  51150 non-null  object
 14  regime_trabalho          76661 non-nul

In [7]:
objetos_contratos_df = pd.read_sql(SQL_OBJETOS_CONTRATOS, conn)
objetos_contratos_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26123 entries, 0 to 26122
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sigla_lotacao    26123 non-null  object
 1   objeto_contrato  26123 non-null  object
dtypes: object(2)
memory usage: 408.3+ KB


In [ ]:
cursos_df = pd.read_sql(SQL_CURSOS, conn)
cursos_df.info(verbose=True, show_counts=True)

In [ ]:
conn_duckdb = duckdb.connect(':memory:')
sql = """
WITH cte_objetos AS (
    SELECT  a.sigla_lotacao
    ,       LIST(DISTINCT a.objeto_contrato) AS objetos_contratos
    FROM    objetos_contratos_df AS a
    GROUP   BY a.sigla_lotacao    
)
, cte_cursos AS (
    SELECT  a.chave_usuario
    ,       LIST(DISTINCT a.curso) AS cursos
    FROM    cursos_df AS a
    GROUP   BY a.chave_usuario    
)
SELECT  a.chave_usuario
,       a.tipo_usuario
,       a.centro_custo
,       a.lotacao_topo
,       a.sigla_lotacao
,       a.nome_lotacao
,       a.cargo
,       a.enfase
,       a.funcao
,       a.sindicato
,       a.area_rh
,       a.imovel
,       a.local_negocio
,       a.grupo_prestacao_servico
,       a.regime_trabalho
,       a.empresa_contrato
,       a.tipo_localizacao
,       a.status_cracha
,       a.situacao_cracha
,       COALESCE(b.objetos_contratos, []) AS objetos_contratos
,       COALESCE(c.cursos, [])            AS cursos
FROM    _birh_df a
        --
        LEFT OUTER JOIN cte_objetos b
        ON  a.sigla_lotacao         = b.sigla_lotacao
        --
        LEFT OUTER JOIN cte_cursos c
        ON  a.chave_usuario         = c.chave_usuario
"""
birh_df = conn_duckdb.execute(sql).fetchdf()
birh_df.info(verbose=True, show_counts=True)

Salvando o conjunto de dados como um arquivo parquet para poder posteriormente consultá-lo usando a biblioteca **DuckDB**

In [ ]:
birh_df.to_parquet(os.environ['BIRH_DATA'])

In [ ]:
print(f"arquivo do BIRH gerado -> {os.environ['BIRH_DATA']}")